# CapiPort - PORTFOLIO OPTIMISATION

    Two things to consider for Portfolio Optimisation:
    
        1) Minimising Risk
        2) Maximising Return

    Basic process of Portfolio Optimisation:
        
        1) Select the Asset class to work on.
            1.1) Asset Class choosen - Equity (Stocks)
        2) Select the Companies which you want to use to build a Portfolio.
            2.1) Companies choosen - 
                2.1.1) Tata Power - TATAPOWER.NS
                2.1.2) Tata Motors - TATAMOTORS.NS
                2.1.3) Tata Steel - TATASTEEL.NS
                2.1.4) Zomato - ZOMATO.NS
                2.1.5) NHPC - NHPC.NS
                2.1.6) NCC - NCC.NS
                2.1.7) IREDA - IREDA.NS
                2.1.8) IRCON - IRCON.NS
        3) To try various Statistical Methods relating to Portfolio Optimisation.
            3.1) Method 1 - Result
            3.2) Method 2 - Result
        4) You will obtain Weigths or Percentages of Portfolio to invest.
            4.1) Method 1 - Weights
            4.2) Method 2 - Weights
        5) Testing the Portfolio for the future.
            5.1) Method 1 - Result
            5.2) Method 2 - Result
        6) Final Result

## Importing the Libraries

In [15]:
import yfinance as yf


from scipy.optimize import minimize



import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Get Stock Data

In [16]:
def get_historical_returns(tickers, start_date, end_date):
    """
    Fetch historical returns data for the given tickers.

    Args:
    - tickers: list of strings, tickers of assets
    - start_date: string, start date in the format 'YYYY-MM-DD'
    - end_date: string, end date in the format 'YYYY-MM-DD'

    Returns:
    - pandas DataFrame, historical returns data
    """
    data = yf.download(tickers, start=start_date, end=end_date)['Adj Close']
    returns = data.pct_change().dropna()
    return returns

def get_risk_free_rate_india():
    """
    Get the risk-free rate for the Indian market using the yield of the 10-year Indian Government Bond.

    Returns:
    - float, risk-free rate
    """
    # Ticker symbol for the 10-year Indian Government Bond yield
    bond_ticker = 'INR=X'  # You can replace this with the actual ticker symbol for the bond

    # Fetch the bond data
    bond_data = yf.Ticker(bond_ticker)

    # Get the latest yield
    risk_free_rate_india = bond_data.history(period='1d')['Close'][-1] / 100
    return risk_free_rate_india

In [26]:
equity_list = ["TATAPOWER.NS", "TATAMOTORS.NS", "TATASTEEL.NS", "ZOMATO.NS", "NHPC.NS", "NCC.NS", "IREDA.NS", "IRCON.NS"]

equity_data = get_historical_returns(equity_list, "1900-01-01", "2024-03-04")

risk_free_rate = get_risk_free_rate_india()

[*********************100%%**********************]  8 of 8 completed
/var/folders/jx/_r4pg95j3pzdd581p_wql9pc0000gn/T/ipykernel_17224/562231645.py:14: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Call ffill before calling pct_change to retain current behavior and silence this warning.
  returns = data.pct_change().dropna()
/var/folders/jx/_r4pg95j3pzdd581p_wql9pc0000gn/T/ipykernel_17224/4250087689.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  risk_free_rate_india = bond_data.history(period='1d')['Close'][-1] / 100


In [28]:
def sharpe_ratio(weights, returns, risk_free_rate):
    """
    Calculate the Sharpe Ratio of a portfolio.

    Args:
    - weights: array-like, weights of assets in the portfolio
    - returns: pandas DataFrame, historical returns of assets
    - risk_free_rate: float, risk-free rate of return

    Returns:
    - float, Sharpe Ratio of the portfolio
    """
    portfolio_return = np.sum(weights * returns.mean() * 252)  # 252 trading days in a year
    portfolio_std_dev = np.sqrt(np.dot(weights.T, np.dot(returns.cov() * 252, weights)))
    sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_std_dev
    return -sharpe_ratio  # Minimize negative Sharpe Ratio for maximization



def optimize_portfolio(returns, risk_free_rate):
    """
    Optimize portfolio to maximize the Sharpe Ratio.

    Args:
    - returns: pandas DataFrame, historical returns of assets
    - risk_free_rate: float, risk-free rate of return

    Returns:
    - array, optimal weights of assets in the portfolio
    """
    num_assets = len(returns.columns)
    initial_weights = np.array([1 / num_assets] * num_assets)
    bounds = [(0, 1)] * num_assets  # Bounds for asset weights (0 <= weight <= 1)
    constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})  # Sum of weights equals 1 constraint

    optimized_result = minimize(sharpe_ratio, initial_weights, args=(returns, risk_free_rate),
                                method='SLSQP', bounds=bounds, constraints=constraints)

    return optimized_result.x

In [33]:
optimal_weights = optimize_portfolio(equity_data, risk_free_rate)
for i,j in zip(equity_list, optimal_weights):
    print(i, " : ", j)

TATAPOWER.NS  :  0.0
TATAMOTORS.NS  :  0.236548217499089
TATASTEEL.NS  :  0.17568900379556238
ZOMATO.NS  :  0.07826482194498546
NHPC.NS  :  0.21116461887342103
NCC.NS  :  0.0
IREDA.NS  :  0.0
IRCON.NS  :  0.2983333378869751


## <u>STEPS FOR IMPLEMENTING<u>

    1) IMPORTING THE LIBRARIES
    2) TWEETS EXTRACTION FROM STOCKNET
    3) TWITTER DATA PRE-PROCESSING
    4) ZERO-SHOT SENTIMENT CLASSIFICATION
    5) FEATURE ENGINEERING OF TWEETS SENTIMENT VALUES
        5.1) Number of Tweets for each individual days
        5.2) Average of Emotion for each individual days
        5.3) Median of Sentiment for each Single Day
        5.4) Maximum Sentiment Value for each Single day
        5.5) Minimum Sentiment Value for Each Single Day
        5.6) Combining all the dataframes
    6) STOCK DATA FROM STOCKNET
    7) STOCK DATA AND FEATURE ENGINEERED SENTIMENT VALUES MERGING STEP
    9) WITH SENTIMENT
        9.1) DATASET PREPARATION FOR TRAINING
        9.2) TRAINING
        9.3) EVALUATING
        9.4) GRAPHS AND METRICS